# Mean-Variance Portfolio Optimization
Before using, select `File` in the ribbon above and then select `Save a copy in Drive`.  This way, your interactions won't carry over to other users.

Feel free to email rgilland@hbs.edu if you have any questions or if you discover any bugs with the program.

## Importing Libraries
This worksheet uses some libraries that are either non-existent or outdated in the default Google Colab kernel.  To make sure this sheet can be used properly, run the below code by clicking on the play button below, then restart and execute the runtime by selecting `Runtime` in the ribbon above, then selecting `Restart and run all`. Then you will be able to interact with the optimizer (change inputs for the optimizer, run it, etc.)

There are thousands of pre-existing libraries for Python.  Rather than building everything from scratch, this optimizer loads a handful of these libraries using the code below.

In [1]:
from pypfopt import efficient_frontier, expected_returns, risk_models, plotting
from IPython.display import display, Markdown, clear_output
import ipywidgets as wg

import os
import copy
import math
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import cycler
from scipy import optimize

plt.rc('figure', facecolor='#282a2e', edgecolor='none')
plt.rc('axes', facecolor='#3a404a', edgecolor='none', labelcolor='white')
mpl.rc('axes', prop_cycle=cycler.cycler(color=['#ff7f0e', 'white']))
plt.rc('text', color='white')
plt.rc('xtick', color='white')
plt.rc('ytick', color='white')
plt.rc('patch', edgecolor='white')


## Initializing Asset Classes
Use `Add Asset Class` to add a new asset class to the table (and correlation matrix), then update return, standard deviation, etc. as you see fit.  Return and standard deviation are in percentage terms.

Use `Default Setup` to load the parameters from the HBS Case "Investment Policy at the Hewlett Foundation" (HBS Case 205-126). Otherwise feel free to enter your own asset classes or remove them using the appropriate buttons below. You can also change the numbers in each cell. Note that expected returns, standard deviations, and weights shoudl all be in percent. This model assumes you enter moments for continuously compounded returns (i.e., geometric mean and standard deviations).

In [2]:
# @title
"""
WARNING: This is probably the ugliest code I've ever written and nothing is commented
"""
asset_count = 2

btn_port_plot = wg.Button(description="Fit Portfolio")
btn_weights = wg.Button(description="Optimize Weights", disabled=True)

name = wg.Text(
    value='Item ' + str(asset_count),
    placeholder='Enter an asset class name',
    description='New Asset:',
    disabled=False,
    layout=wg.Layout(width='15%'),
)
geo_ret = wg.FloatText(value=8.0, layout=wg.Layout(width='10%'))
stdev = wg.BoundedFloatText(value=15.9, min=0.0, layout=wg.Layout(width='10%'))
min_weight = wg.FloatText(value=-100.0, layout=wg.Layout(width='10%'))
max_weight = wg.FloatText(value=100.0, layout=wg.Layout(width='10%'))
new_asset = wg.HBox([name, geo_ret, stdev, min_weight, max_weight])

header_name = wg.Button(
    description='Name',
    disabled=True,
    button_style='',
    tooltip='',
    icon='',
    layout=wg.Layout(width='15%'),
)
header_return = wg.Button(
    description='Return',
    disabled=True,
    button_style='',
    tooltip='',
    icon='',
    layout=wg.Layout(width='10%'),
)
header_stdev = wg.Button(
    description='StDev',
    disabled=True,
    button_style='',
    tooltip='',
    icon='',
    layout=wg.Layout(width='10%'),
)
header_wmin = wg.Button(
    description='Min Weight',
    disabled=True,
    button_style='',
    tooltip='',
    icon='',
    layout=wg.Layout(width='10%'),
)
header_wmax = wg.Button(
    description='Max Weight',
    disabled=True,
    button_style='',
    tooltip='',
    icon='',
    layout=wg.Layout(width='10%'),
)

header = wg.HBox([header_name, header_return, header_stdev, header_wmin, header_wmax])

assets = wg.VBox(
    [
        header,
        wg.HBox(
            [
                wg.Text("Item 0", layout=wg.Layout(width='15%')),
                wg.FloatText(value=8.0, layout=wg.Layout(width='10%')),
                wg.BoundedFloatText(value=15.9, min=0.0, layout=wg.Layout(width='10%')),
                wg.FloatText(value=-100, layout=wg.Layout(width='10%')),
                wg.FloatText(value=100, layout=wg.Layout(width='10%')),
            ]
        ),
        wg.HBox(
            [
                wg.Text("Item 1", layout=wg.Layout(width='15%')),
                wg.FloatText(value=8.7, layout=wg.Layout(width='10%')),
                wg.BoundedFloatText(value=19.2, min=0.0, layout=wg.Layout(width='10%')),
                wg.FloatText(value=-100, layout=wg.Layout(width='10%')),
                wg.FloatText(value=100, layout=wg.Layout(width='10%')),
            ]
        ),
    ]
)

int_corr_matrix = wg.VBox(
    [
        wg.HBox(
            [
                wg.Button(
                    description='',
                    disabled=True,
                    button_style='',
                    tooltip='',
                    icon='',
                    layout=wg.Layout(width='5%'),
                ),
                wg.Button(
                    description=assets.children[1].children[0].value,
                    disabled=True,
                    button_style='',
                    tooltip='',
                    icon='',
                    layout=wg.Layout(width='5%'),
                ),
                wg.Button(
                    description=assets.children[2].children[0].value,
                    disabled=True,
                    button_style='',
                    tooltip='',
                    icon='',
                    layout=wg.Layout(width='5%'),
                ),
            ]
        ),
        wg.HBox(
            [
                wg.Button(
                    description=assets.children[1].children[0].value,
                    disabled=True,
                    button_style='',
                    tooltip='',
                    icon='',
                    layout=wg.Layout(width='5%'),
                ),
                wg.BoundedFloatText(
                    value=1.000,
                    min=-1.000,
                    max=1.000,
                    step=0.01,
                    layout=wg.Layout(width='5%'),
                ),
                wg.BoundedFloatText(
                    value=0.750,
                    min=-1.000,
                    max=1.000,
                    step=0.01,
                    layout=wg.Layout(width='5%'),
                ),
            ]
        ),
        wg.HBox(
            [
                wg.Button(
                    description=assets.children[2].children[0].value,
                    disabled=True,
                    button_style='',
                    tooltip='',
                    icon='',
                    layout=wg.Layout(width='5%'),
                ),
                wg.BoundedFloatText(
                    value=0.750,
                    min=-1.000,
                    max=1.000,
                    step=0.01,
                    layout=wg.Layout(width='5%'),
                ),
                wg.BoundedFloatText(
                    value=1.000,
                    min=-1.000,
                    max=1.000,
                    step=0.01,
                    layout=wg.Layout(width='5%'),
                ),
            ]
        ),
    ]
)

corr_links = []
for i in range(asset_count):
    for j in range(asset_count):
        if i != j:
            corr_links.append(
                wg.link(
                    (int_corr_matrix.children[i + 1].children[j + 1], 'value'),
                    (int_corr_matrix.children[j + 1].children[i + 1], 'value'),
                )
            )
name_links = []
for i in range(asset_count):
    name_links.append(
        wg.link(
            (assets.children[i + 1].children[0], 'value'),
            (int_corr_matrix.children[0].children[i + 1], 'description'),
        )
    )
    name_links.append(
        wg.link(
            (assets.children[i + 1].children[0], 'value'),
            (int_corr_matrix.children[i + 1].children[0], 'description'),
        )
    )

for i in range(asset_count):
    for j in range(asset_count):
        if j >= i:
            int_corr_matrix.children[i + 1].children[j + 1].disabled = True
add_btn = wg.Button(description="Add Asset Class")
rem_btn = wg.Button(description="Remove Asset Class", disabled=True)
default_btn = wg.Button(description="Default Setup")

name_links = []
for i in range(asset_count):
    name_links.append(
        wg.link(
            (assets.children[i + 1].children[0], 'value'),
            (int_corr_matrix.children[0].children[i + 1], 'description'),
        )
    )
    name_links.append(
        wg.link(
            (assets.children[i + 1].children[0], 'value'),
            (int_corr_matrix.children[i + 1].children[0], 'description'),
        )
    )

def on_val_change(change):
  btn_weights.disabled = True

val_observe = []
for i in range(asset_count):
  for j in range(asset_count):
    val_observe.append(int_corr_matrix.children[i+1].children[j+1].observe(on_val_change))

for i in range(asset_count):
  for j in range(4):
    val_observe.append(assets.children[i+1].children[j+1].observe(on_val_change))

def on_add_click(b):
    global asset_count, int_corr_matrix, corr_matrix, cov_matrix, exp_returns, stdevs

    prev_asset = wg.HBox(
        [
            wg.Text(value=name.value, layout=wg.Layout(width='15%')),
            wg.FloatText(value=geo_ret.value, layout=wg.Layout(width='10%')),
            wg.BoundedFloatText(value=stdev.value, min=0.0, layout=wg.Layout(width='10%')),
            wg.FloatText(value=min_weight.value, layout=wg.Layout(width='10%')),
            wg.FloatText(value=max_weight.value, layout=wg.Layout(width='10%')),
        ]
    )
    assets.children = tuple(list(assets.children) + [prev_asset])

    new_int_corr_matrix = np.eye(asset_count + 1, asset_count + 1)
    for i in range(asset_count):
        for j in range(asset_count):
            new_int_corr_matrix[i, j] = (
                int_corr_matrix.children[i + 1].children[j + 1].value
            )
    asset_count += 1
    name.value = 'Item ' + str(asset_count)

    int_corr_matrix.children = tuple(
        list(
            wg.VBox(
                [
                    wg.HBox(
                        [
                            wg.Button(
                                description="",
                                disabled=True,
                                button_style="",
                                tooltip="",
                                icon="",
                                layout=wg.Layout(width='5%'),
                            )
                        ]
                        + [
                            wg.Button(
                                description=assets.children[j + 1].children[0].value,
                                disabled=True,
                                button_style="",
                                tooltip="",
                                icon="",
                                layout=wg.Layout(width='5%'),
                            )
                            for j in range(asset_count)
                        ]
                    )
                ]
                + [
                    wg.HBox(
                        [
                            wg.Button(
                                description=assets.children[i + 1].children[0].value,
                                disabled=True,
                                button_style="",
                                tooltip="",
                                icon="",
                                layout=wg.Layout(width='5%'),
                            )
                        ]
                        + [
                            wg.BoundedFloatText(
                                value=new_int_corr_matrix[i, j],
                                min=-1.000,
                                max=1.000,
                                step=0.01,
                                layout=wg.Layout(width='5%'),
                            )
                            for j in range(asset_count)
                        ]
                    )
                    for i in range(asset_count)
                ]
            ).children
        )
    )
    corr_links = []
    for i in range(asset_count):
        for j in range(asset_count):
            if i != j:
                corr_links.append(
                    wg.link(
                        (int_corr_matrix.children[i + 1].children[j + 1], 'value'),
                        (int_corr_matrix.children[j + 1].children[i + 1], 'value'),
                    )
                )
    name_links = []
    for i in range(asset_count):
        name_links.append(
            wg.link(
                (assets.children[i + 1].children[0], 'value'),
                (int_corr_matrix.children[0].children[i + 1], 'description'),
            )
        )
        name_links.append(
            wg.link(
                (assets.children[i + 1].children[0], 'value'),
                (int_corr_matrix.children[i + 1].children[0], 'description'),
            )
        )
    for i in range(asset_count):
        for j in range(asset_count):
            if j >= i:
                int_corr_matrix.children[i + 1].children[j + 1].disabled = True
    
    val_observe = []
    for i in range(asset_count):
      for j in range(asset_count):
        val_observe.append(int_corr_matrix.children[i+1].children[j+1].observe(on_val_change))

    for i in range(asset_count):
      for j in range(4):
        val_observe.append(assets.children[i+1].children[j+1].observe(on_val_change))

    if asset_count > 2:
        rem_btn.disabled = False
    else:
        rem_btn.disabled = True
    
    btn_weights.disabled=True


def on_rem_click(b):
    global asset_count, int_corr_matrix, corr_matrix, cov_matrix, exp_returns, stdevs

    assets.children = tuple(list(assets.children)[:-1])

    new_int_corr_matrix = np.zeros((asset_count - 1, asset_count - 1))
    for i in range(asset_count - 1):
        for j in range(asset_count - 1):
            new_int_corr_matrix[i, j] = (
                int_corr_matrix.children[i + 1].children[j + 1].value
            )
    asset_count -= 1
    name.value = 'Item ' + str(asset_count)

    int_corr_matrix.children = tuple(
        list(
            wg.VBox(
                [
                    wg.HBox(
                        [
                            wg.Button(
                                description="",
                                disabled=True,
                                button_style="",
                                tooltip="",
                                icon="",
                                layout=wg.Layout(width='5%'),
                            )
                        ]
                        + [
                            wg.Button(
                                description=assets.children[j + 1].children[0].value,
                                disabled=True,
                                button_style="",
                                tooltip="",
                                icon="",
                                layout=wg.Layout(width='5%'),
                            )
                            for j in range(asset_count)
                        ]
                    )
                ]
                + [
                    wg.HBox(
                        [
                            wg.Button(
                                description=assets.children[i + 1].children[0].value,
                                disabled=True,
                                button_style="",
                                tooltip="",
                                icon="",
                                layout=wg.Layout(width='5%'),
                            )
                        ]
                        + [
                            wg.BoundedFloatText(
                                value=new_int_corr_matrix[i, j],
                                min=-1.000,
                                max=1.000,
                                step=0.01,
                                layout=wg.Layout(width='5%'),
                            )
                            for j in range(asset_count)
                        ]
                    )
                    for i in range(asset_count)
                ]
            ).children
        )
    )

    corr_links = []
    for i in range(asset_count):
        for j in range(asset_count):
            if i != j:
                corr_links.append(
                    wg.link(
                        (int_corr_matrix.children[i + 1].children[j + 1], 'value'),
                        (int_corr_matrix.children[j + 1].children[i + 1], 'value'),
                    )
                )
    name_links = []
    for i in range(asset_count):
        name_links.append(
            wg.link(
                (assets.children[i + 1].children[0], 'value'),
                (int_corr_matrix.children[0].children[i + 1], 'description'),
            )
        )
        name_links.append(
            wg.link(
                (assets.children[i + 1].children[0], 'value'),
                (int_corr_matrix.children[i + 1].children[0], 'description'),
            )
        )
    for i in range(asset_count):
        for j in range(asset_count):
            if j >= i:
                int_corr_matrix.children[i + 1].children[j + 1].disabled = True

    val_observe = []
    for i in range(asset_count):
      for j in range(asset_count):
        val_observe.append(int_corr_matrix.children[i+1].children[j+1].observe(on_val_change))

    for i in range(asset_count):
      for j in range(4):
        val_observe.append(assets.children[i+1].children[j+1].observe(on_val_change))

    if asset_count > 2:
        rem_btn.disabled = False
    else:
        rem_btn.disabled = True

    btn_weights.disabled=True


def on_default_click(b):
    global asset_count, int_corr_matrix, corr_matrix, cov_matrix, exp_returns, stdevs
    asset_count = 10

    assets.children = (
        wg.HBox(
            children=(
                wg.Button(
                    description='Name',
                    disabled=True,
                    button_style='',
                    tooltip='',
                    icon='',
                    layout=wg.Layout(width='15%'),
                ),
                wg.Button(
                    description='Return',
                    disabled=True,
                    button_style='',
                    tooltip='',
                    icon='',
                    layout=wg.Layout(width='10%'),
                ),
                wg.Button(
                    description='StDev',
                    disabled=True,
                    button_style='',
                    tooltip='',
                    icon='',
                    layout=wg.Layout(width='10%'),
                ),
                wg.Button(
                    description='Min Weight',
                    disabled=True,
                    button_style='',
                    tooltip='',
                    icon='',
                    layout=wg.Layout(width='10%'),
                ),
                wg.Button(
                    description='Max Weight',
                    disabled=True,
                    button_style='',
                    tooltip='',
                    icon='',
                    layout=wg.Layout(width='10%'),
                ),
            )
        ),
        wg.HBox(
            children=(
                wg.Text(value='Domestic Equity', layout=wg.Layout(width='15%')),
                wg.FloatText(value=8.0, layout=wg.Layout(width='10%')),
                wg.BoundedFloatText(value=15.9, min=0.0, layout=wg.Layout(width='10%')),
                wg.FloatText(value=-100.0, layout=wg.Layout(width='10%')),
                wg.FloatText(value=100.0, layout=wg.Layout(width='10%')),
            )
        ),
        wg.HBox(
            children=(
                wg.Text(value='International Equity', layout=wg.Layout(width='15%')),
                wg.FloatText(value=8.7, layout=wg.Layout(width='10%')),
                wg.BoundedFloatText(value=19.2, min=0.0, layout=wg.Layout(width='10%')),
                wg.FloatText(value=-100.0, layout=wg.Layout(width='10%')),
                wg.FloatText(value=100.0, layout=wg.Layout(width='10%')),
            )
        ),
        wg.HBox(
            children=(
                wg.Text(value='Emerging Markets', layout=wg.Layout(width='15%')),
                wg.FloatText(value=9.7, layout=wg.Layout(width='10%')),
                wg.BoundedFloatText(value=30.0, min=0.0, layout=wg.Layout(width='10%')),
                wg.FloatText(value=-100.0, layout=wg.Layout(width='10%')),
                wg.FloatText(value=100.0, layout=wg.Layout(width='10%')),
            )
        ),
        wg.HBox(
            children=(
                wg.Text(value='Private Equity', layout=wg.Layout(width='15%')),
                wg.FloatText(value=11.0, layout=wg.Layout(width='10%')),
                wg.BoundedFloatText(value=38.0, min=0.0, layout=wg.Layout(width='10%')),
                wg.FloatText(value=-100.0, layout=wg.Layout(width='10%')),
                wg.FloatText(value=100.0, layout=wg.Layout(width='10%')),
            )
        ),
        wg.HBox(
            children=(
                wg.Text(value='Real Assets', layout=wg.Layout(width='15%')),
                wg.FloatText(value=6.7, layout=wg.Layout(width='10%')),
                wg.BoundedFloatText(value=13.0, min=0.0, layout=wg.Layout(width='10%')),
                wg.FloatText(value=-100.0, layout=wg.Layout(width='10%')),
                wg.FloatText(value=100.0, layout=wg.Layout(width='10%')),
            )
        ),
        wg.HBox(
            children=(
                wg.Text(value='Absolute Return', layout=wg.Layout(width='15%')),
                wg.FloatText(value=5.5, layout=wg.Layout(width='10%')),
                wg.BoundedFloatText(value=7.5, min=0.0, layout=wg.Layout(width='10%')),
                wg.FloatText(value=-100.0, layout=wg.Layout(width='10%')),
                wg.FloatText(value=100.0, layout=wg.Layout(width='10%')),
            )
        ),
        wg.HBox(
            children=(
                wg.Text(value='Bonds', layout=wg.Layout(width='15%')),
                wg.FloatText(value=4.4, layout=wg.Layout(width='10%')),
                wg.BoundedFloatText(value=5.0, min=0.0, layout=wg.Layout(width='10%')),
                wg.FloatText(value=-100.0, layout=wg.Layout(width='10%')),
                wg.FloatText(value=100.0, layout=wg.Layout(width='10%')),
            )
        ),
        wg.HBox(
            children=(
                wg.Text(value='TIPS', layout=wg.Layout(width='15%')),
                wg.FloatText(value=4.0, layout=wg.Layout(width='10%')),
                wg.BoundedFloatText(value=4.0, min=0.0, layout=wg.Layout(width='10%')),
                wg.FloatText(value=-100.0, layout=wg.Layout(width='10%')),
                wg.FloatText(value=100.0, layout=wg.Layout(width='10%')),
            )
        ),
        wg.HBox(
            children=(
                wg.Text(value='High-Yield', layout=wg.Layout(width='15%')),
                wg.FloatText(value=6.0, layout=wg.Layout(width='10%')),
                wg.BoundedFloatText(value=8.5, min=0.0, layout=wg.Layout(width='10%')),
                wg.FloatText(value=-100.0, layout=wg.Layout(width='10%')),
                wg.FloatText(value=100.0, layout=wg.Layout(width='10%')),
            )
        ),
        wg.HBox(
            children=(
                wg.Text(value='Cash', layout=wg.Layout(width='15%')),
                wg.FloatText(value=3.5, layout=wg.Layout(width='10%')),
                wg.BoundedFloatText(value=0.0, min=0.0, layout=wg.Layout(width='10%')),
                wg.FloatText(value=-20.0, layout=wg.Layout(width='10%')),
                wg.FloatText(value=100.0, layout=wg.Layout(width='10%')),
            )
        ),
    )
    int_corr_matrix.children = tuple(
        list(
            (
                wg.HBox(
                    children=(
                        wg.Button(
                            description='',
                            disabled=True,
                            button_style='',
                            tooltip='',
                            icon='',
                            layout=wg.Layout(width='5%'),
                        ),
                        wg.Button(
                            description='Domestic Equity',
                            disabled=True,
                            button_style='',
                            tooltip='',
                            icon='',
                            layout=wg.Layout(width='5%'),
                        ),
                        wg.Button(
                            description='International Equity',
                            disabled=True,
                            button_style='',
                            tooltip='',
                            icon='',
                            layout=wg.Layout(width='5%'),
                        ),
                        wg.Button(
                            description='Emerging Markets',
                            disabled=True,
                            button_style='',
                            tooltip='',
                            icon='',
                            layout=wg.Layout(width='5%'),
                        ),
                        wg.Button(
                            description='Private Equity',
                            disabled=True,
                            button_style='',
                            tooltip='',
                            icon='',
                            layout=wg.Layout(width='5%'),
                        ),
                        wg.Button(
                            description='Real Assets',
                            disabled=True,
                            button_style='',
                            tooltip='',
                            icon='',
                            layout=wg.Layout(width='5%'),
                        ),
                        wg.Button(
                            description='Absolute Return',
                            disabled=True,
                            button_style='',
                            tooltip='',
                            icon='',
                            layout=wg.Layout(width='5%'),
                        ),
                        wg.Button(
                            description='Bonds',
                            disabled=True,
                            button_style='',
                            tooltip='',
                            icon='',
                            layout=wg.Layout(width='5%'),
                        ),
                        wg.Button(
                            description='TIPS',
                            disabled=True,
                            button_style='',
                            tooltip='',
                            icon='',
                            layout=wg.Layout(width='5%'),
                        ),
                        wg.Button(
                            description='High-Yield',
                            disabled=True,
                            button_style='',
                            tooltip='',
                            icon='',
                            layout=wg.Layout(width='5%'),
                        ),
                        wg.Button(
                            description='Cash',
                            disabled=True,
                            button_style='',
                            tooltip='',
                            icon='',
                            layout=wg.Layout(width='5%'),
                        ),
                    )
                ),
                wg.HBox(
                    children=(
                        wg.Button(
                            description='Domestic Equity',
                            disabled=True,
                            button_style='',
                            tooltip='',
                            icon='',
                            layout=wg.Layout(width='5%'),
                        ),
                        wg.BoundedFloatText(
                            value=1.0,
                            layout=wg.Layout(width='5%'),
                            max=1.0,
                            min=-1.0,
                            step=0.01,
                        ),
                        wg.BoundedFloatText(
                            value=0.75,
                            layout=wg.Layout(width='5%'),
                            max=1.0,
                            min=-1.0,
                            step=0.01,
                        ),
                        wg.BoundedFloatText(
                            value=0.6,
                            layout=wg.Layout(width='5%'),
                            max=1.0,
                            min=-1.0,
                            step=0.01,
                        ),
                        wg.BoundedFloatText(
                            value=0.6,
                            layout=wg.Layout(width='5%'),
                            max=1.0,
                            min=-1.0,
                            step=0.01,
                        ),
                        wg.BoundedFloatText(
                            value=0.4,
                            layout=wg.Layout(width='5%'),
                            max=1.0,
                            min=-1.0,
                            step=0.01,
                        ),
                        wg.BoundedFloatText(
                            value=0.35,
                            layout=wg.Layout(width='5%'),
                            max=1.0,
                            min=-1.0,
                            step=0.01,
                        ),
                        wg.BoundedFloatText(
                            value=0.2,
                            layout=wg.Layout(width='5%'),
                            max=1.0,
                            min=-1.0,
                            step=0.01,
                        ),
                        wg.BoundedFloatText(
                            value=0.0,
                            layout=wg.Layout(width='5%'),
                            max=1.0,
                            min=-1.0,
                            step=0.01,
                        ),
                        wg.BoundedFloatText(
                            value=0.55,
                            layout=wg.Layout(width='5%'),
                            max=1.0,
                            min=-1.0,
                            step=0.01,
                        ),
                        wg.BoundedFloatText(
                            value=0.0,
                            layout=wg.Layout(width='5%'),
                            max=1.0,
                            min=-1.0,
                            step=0.01,
                        ),
                    )
                ),
                wg.HBox(
                    children=(
                        wg.Button(
                            description='International Equity',
                            disabled=True,
                            button_style='',
                            tooltip='',
                            icon='',
                            layout=wg.Layout(width='5%'),
                        ),
                        wg.BoundedFloatText(
                            value=0.75,
                            layout=wg.Layout(width='5%'),
                            max=1.0,
                            min=-1.0,
                            step=0.01,
                        ),
                        wg.BoundedFloatText(
                            value=1.0,
                            layout=wg.Layout(width='5%'),
                            max=1.0,
                            min=-1.0,
                            step=0.01,
                        ),
                        wg.BoundedFloatText(
                            value=0.65,
                            layout=wg.Layout(width='5%'),
                            max=1.0,
                            min=-1.0,
                            step=0.01,
                        ),
                        wg.BoundedFloatText(
                            value=0.55,
                            layout=wg.Layout(width='5%'),
                            max=1.0,
                            min=-1.0,
                            step=0.01,
                        ),
                        wg.BoundedFloatText(
                            value=0.45,
                            layout=wg.Layout(width='5%'),
                            max=1.0,
                            min=-1.0,
                            step=0.01,
                        ),
                        wg.BoundedFloatText(
                            value=0.25,
                            layout=wg.Layout(width='5%'),
                            max=1.0,
                            min=-1.0,
                            step=0.01,
                        ),
                        wg.BoundedFloatText(
                            value=0.2,
                            layout=wg.Layout(width='5%'),
                            max=1.0,
                            min=-1.0,
                            step=0.01,
                        ),
                        wg.BoundedFloatText(
                            value=0.1,
                            layout=wg.Layout(width='5%'),
                            max=1.0,
                            min=-1.0,
                            step=0.01,
                        ),
                        wg.BoundedFloatText(
                            value=0.5,
                            layout=wg.Layout(width='5%'),
                            max=1.0,
                            min=-1.0,
                            step=0.01,
                        ),
                        wg.BoundedFloatText(
                            value=0.0,
                            layout=wg.Layout(width='5%'),
                            max=1.0,
                            min=-1.0,
                            step=0.01,
                        ),
                    )
                ),
                wg.HBox(
                    children=(
                        wg.Button(
                            description='Emerging Markets',
                            disabled=True,
                            button_style='',
                            tooltip='',
                            icon='',
                            layout=wg.Layout(width='5%'),
                        ),
                        wg.BoundedFloatText(
                            value=0.6,
                            layout=wg.Layout(width='5%'),
                            max=1.0,
                            min=-1.0,
                            step=0.01,
                        ),
                        wg.BoundedFloatText(
                            value=0.65,
                            layout=wg.Layout(width='5%'),
                            max=1.0,
                            min=-1.0,
                            step=0.01,
                        ),
                        wg.BoundedFloatText(
                            value=1.0,
                            layout=wg.Layout(width='5%'),
                            max=1.0,
                            min=-1.0,
                            step=0.01,
                        ),
                        wg.BoundedFloatText(
                            value=0.5,
                            layout=wg.Layout(width='5%'),
                            max=1.0,
                            min=-1.0,
                            step=0.01,
                        ),
                        wg.BoundedFloatText(
                            value=0.4,
                            layout=wg.Layout(width='5%'),
                            max=1.0,
                            min=-1.0,
                            step=0.01,
                        ),
                        wg.BoundedFloatText(
                            value=0.4,
                            layout=wg.Layout(width='5%'),
                            max=1.0,
                            min=-1.0,
                            step=0.01,
                        ),
                        wg.BoundedFloatText(
                            value=0.2,
                            layout=wg.Layout(width='5%'),
                            max=1.0,
                            min=-1.0,
                            step=0.01,
                        ),
                        wg.BoundedFloatText(
                            value=0.1,
                            layout=wg.Layout(width='5%'),
                            max=1.0,
                            min=-1.0,
                            step=0.01,
                        ),
                        wg.BoundedFloatText(
                            value=0.55,
                            layout=wg.Layout(width='5%'),
                            max=1.0,
                            min=-1.0,
                            step=0.01,
                        ),
                        wg.BoundedFloatText(
                            value=0.0,
                            layout=wg.Layout(width='5%'),
                            max=1.0,
                            min=-1.0,
                            step=0.01,
                        ),
                    )
                ),
                wg.HBox(
                    children=(
                        wg.Button(
                            description='Private Equity',
                            disabled=True,
                            button_style='',
                            tooltip='',
                            icon='',
                            layout=wg.Layout(width='5%'),
                        ),
                        wg.BoundedFloatText(
                            value=0.6,
                            layout=wg.Layout(width='5%'),
                            max=1.0,
                            min=-1.0,
                            step=0.01,
                        ),
                        wg.BoundedFloatText(
                            value=0.55,
                            layout=wg.Layout(width='5%'),
                            max=1.0,
                            min=-1.0,
                            step=0.01,
                        ),
                        wg.BoundedFloatText(
                            value=0.5,
                            layout=wg.Layout(width='5%'),
                            max=1.0,
                            min=-1.0,
                            step=0.01,
                        ),
                        wg.BoundedFloatText(
                            value=1.0,
                            layout=wg.Layout(width='5%'),
                            max=1.0,
                            min=-1.0,
                            step=0.01,
                        ),
                        wg.BoundedFloatText(
                            value=0.45,
                            layout=wg.Layout(width='5%'),
                            max=1.0,
                            min=-1.0,
                            step=0.01,
                        ),
                        wg.BoundedFloatText(
                            value=0.3,
                            layout=wg.Layout(width='5%'),
                            max=1.0,
                            min=-1.0,
                            step=0.01,
                        ),
                        wg.BoundedFloatText(
                            value=0.1,
                            layout=wg.Layout(width='5%'),
                            max=1.0,
                            min=-1.0,
                            step=0.01,
                        ),
                        wg.BoundedFloatText(
                            value=0.1,
                            layout=wg.Layout(width='5%'),
                            max=1.0,
                            min=-1.0,
                            step=0.01,
                        ),
                        wg.BoundedFloatText(
                            value=0.4,
                            layout=wg.Layout(width='5%'),
                            max=1.0,
                            min=-1.0,
                            step=0.01,
                        ),
                        wg.BoundedFloatText(
                            value=0.0,
                            layout=wg.Layout(width='5%'),
                            max=1.0,
                            min=-1.0,
                            step=0.01,
                        ),
                    )
                ),
                wg.HBox(
                    children=(
                        wg.Button(
                            description='Real Assets',
                            disabled=True,
                            button_style='',
                            tooltip='',
                            icon='',
                            layout=wg.Layout(width='5%'),
                        ),
                        wg.BoundedFloatText(
                            value=0.4,
                            layout=wg.Layout(width='5%'),
                            max=1.0,
                            min=-1.0,
                            step=0.01,
                        ),
                        wg.BoundedFloatText(
                            value=0.45,
                            layout=wg.Layout(width='5%'),
                            max=1.0,
                            min=-1.0,
                            step=0.01,
                        ),
                        wg.BoundedFloatText(
                            value=0.4,
                            layout=wg.Layout(width='5%'),
                            max=1.0,
                            min=-1.0,
                            step=0.01,
                        ),
                        wg.BoundedFloatText(
                            value=0.45,
                            layout=wg.Layout(width='5%'),
                            max=1.0,
                            min=-1.0,
                            step=0.01,
                        ),
                        wg.BoundedFloatText(
                            value=1.0,
                            layout=wg.Layout(width='5%'),
                            max=1.0,
                            min=-1.0,
                            step=0.01,
                        ),
                        wg.BoundedFloatText(
                            value=0.35,
                            layout=wg.Layout(width='5%'),
                            max=1.0,
                            min=-1.0,
                            step=0.01,
                        ),
                        wg.BoundedFloatText(
                            value=0.4,
                            layout=wg.Layout(width='5%'),
                            max=1.0,
                            min=-1.0,
                            step=0.01,
                        ),
                        wg.BoundedFloatText(
                            value=0.1,
                            layout=wg.Layout(width='5%'),
                            max=1.0,
                            min=-1.0,
                            step=0.01,
                        ),
                        wg.BoundedFloatText(
                            value=0.6,
                            layout=wg.Layout(width='5%'),
                            max=1.0,
                            min=-1.0,
                            step=0.01,
                        ),
                        wg.BoundedFloatText(
                            value=0.0,
                            layout=wg.Layout(width='5%'),
                            max=1.0,
                            min=-1.0,
                            step=0.01,
                        ),
                    )
                ),
                wg.HBox(
                    children=(
                        wg.Button(
                            description='Absolute Return',
                            disabled=True,
                            button_style='',
                            tooltip='',
                            icon='',
                            layout=wg.Layout(width='5%'),
                        ),
                        wg.BoundedFloatText(
                            value=0.35,
                            layout=wg.Layout(width='5%'),
                            max=1.0,
                            min=-1.0,
                            step=0.01,
                        ),
                        wg.BoundedFloatText(
                            value=0.25,
                            layout=wg.Layout(width='5%'),
                            max=1.0,
                            min=-1.0,
                            step=0.01,
                        ),
                        wg.BoundedFloatText(
                            value=0.4,
                            layout=wg.Layout(width='5%'),
                            max=1.0,
                            min=-1.0,
                            step=0.01,
                        ),
                        wg.BoundedFloatText(
                            value=0.3,
                            layout=wg.Layout(width='5%'),
                            max=1.0,
                            min=-1.0,
                            step=0.01,
                        ),
                        wg.BoundedFloatText(
                            value=0.35,
                            layout=wg.Layout(width='5%'),
                            max=1.0,
                            min=-1.0,
                            step=0.01,
                        ),
                        wg.BoundedFloatText(
                            value=1.0,
                            layout=wg.Layout(width='5%'),
                            max=1.0,
                            min=-1.0,
                            step=0.01,
                        ),
                        wg.BoundedFloatText(
                            value=0.3,
                            layout=wg.Layout(width='5%'),
                            max=1.0,
                            min=-1.0,
                            step=0.01,
                        ),
                        wg.BoundedFloatText(
                            value=0.1,
                            layout=wg.Layout(width='5%'),
                            max=1.0,
                            min=-1.0,
                            step=0.01,
                        ),
                        wg.BoundedFloatText(
                            value=0.45,
                            layout=wg.Layout(width='5%'),
                            max=1.0,
                            min=-1.0,
                            step=0.01,
                        ),
                        wg.BoundedFloatText(
                            value=0.0,
                            layout=wg.Layout(width='5%'),
                            max=1.0,
                            min=-1.0,
                            step=0.01,
                        ),
                    )
                ),
                wg.HBox(
                    children=(
                        wg.Button(
                            description='Bonds',
                            disabled=True,
                            button_style='',
                            tooltip='',
                            icon='',
                            layout=wg.Layout(width='5%'),
                        ),
                        wg.BoundedFloatText(
                            value=0.2,
                            layout=wg.Layout(width='5%'),
                            max=1.0,
                            min=-1.0,
                            step=0.01,
                        ),
                        wg.BoundedFloatText(
                            value=0.2,
                            layout=wg.Layout(width='5%'),
                            max=1.0,
                            min=-1.0,
                            step=0.01,
                        ),
                        wg.BoundedFloatText(
                            value=0.2,
                            layout=wg.Layout(width='5%'),
                            max=1.0,
                            min=-1.0,
                            step=0.01,
                        ),
                        wg.BoundedFloatText(
                            value=0.1,
                            layout=wg.Layout(width='5%'),
                            max=1.0,
                            min=-1.0,
                            step=0.01,
                        ),
                        wg.BoundedFloatText(
                            value=0.4,
                            layout=wg.Layout(width='5%'),
                            max=1.0,
                            min=-1.0,
                            step=0.01,
                        ),
                        wg.BoundedFloatText(
                            value=0.3,
                            layout=wg.Layout(width='5%'),
                            max=1.0,
                            min=-1.0,
                            step=0.01,
                        ),
                        wg.BoundedFloatText(
                            value=1.0,
                            layout=wg.Layout(width='5%'),
                            max=1.0,
                            min=-1.0,
                            step=0.01,
                        ),
                        wg.BoundedFloatText(
                            value=-0.1,
                            layout=wg.Layout(width='5%'),
                            max=1.0,
                            min=-1.0,
                            step=0.01,
                        ),
                        wg.BoundedFloatText(
                            value=0.45,
                            layout=wg.Layout(width='5%'),
                            max=1.0,
                            min=-1.0,
                            step=0.01,
                        ),
                        wg.BoundedFloatText(
                            value=0.0,
                            layout=wg.Layout(width='5%'),
                            max=1.0,
                            min=-1.0,
                            step=0.01,
                        ),
                    )
                ),
                wg.HBox(
                    children=(
                        wg.Button(
                            description='TIPS',
                            disabled=True,
                            button_style='',
                            tooltip='',
                            icon='',
                            layout=wg.Layout(width='5%'),
                        ),
                        wg.BoundedFloatText(
                            value=0.0,
                            layout=wg.Layout(width='5%'),
                            max=1.0,
                            min=-1.0,
                            step=0.01,
                        ),
                        wg.BoundedFloatText(
                            value=0.1,
                            layout=wg.Layout(width='5%'),
                            max=1.0,
                            min=-1.0,
                            step=0.01,
                        ),
                        wg.BoundedFloatText(
                            value=0.1,
                            layout=wg.Layout(width='5%'),
                            max=1.0,
                            min=-1.0,
                            step=0.01,
                        ),
                        wg.BoundedFloatText(
                            value=0.1,
                            layout=wg.Layout(width='5%'),
                            max=1.0,
                            min=-1.0,
                            step=0.01,
                        ),
                        wg.BoundedFloatText(
                            value=0.1,
                            layout=wg.Layout(width='5%'),
                            max=1.0,
                            min=-1.0,
                            step=0.01,
                        ),
                        wg.BoundedFloatText(
                            value=0.1,
                            layout=wg.Layout(width='5%'),
                            max=1.0,
                            min=-1.0,
                            step=0.01,
                        ),
                        wg.BoundedFloatText(
                            value=-0.1,
                            layout=wg.Layout(width='5%'),
                            max=1.0,
                            min=-1.0,
                            step=0.01,
                        ),
                        wg.BoundedFloatText(
                            value=1.0,
                            layout=wg.Layout(width='5%'),
                            max=1.0,
                            min=-1.0,
                            step=0.01,
                        ),
                        wg.BoundedFloatText(
                            value=-0.05,
                            layout=wg.Layout(width='5%'),
                            max=1.0,
                            min=-1.0,
                            step=0.01,
                        ),
                        wg.BoundedFloatText(
                            value=0.0,
                            layout=wg.Layout(width='5%'),
                            max=1.0,
                            min=-1.0,
                            step=0.01,
                        ),
                    )
                ),
                wg.HBox(
                    children=(
                        wg.Button(
                            description='High-Yield',
                            disabled=True,
                            button_style='',
                            tooltip='',
                            icon='',
                            layout=wg.Layout(width='5%'),
                        ),
                        wg.BoundedFloatText(
                            value=0.55,
                            layout=wg.Layout(width='5%'),
                            max=1.0,
                            min=-1.0,
                            step=0.01,
                        ),
                        wg.BoundedFloatText(
                            value=0.5,
                            layout=wg.Layout(width='5%'),
                            max=1.0,
                            min=-1.0,
                            step=0.01,
                        ),
                        wg.BoundedFloatText(
                            value=0.55,
                            layout=wg.Layout(width='5%'),
                            max=1.0,
                            min=-1.0,
                            step=0.01,
                        ),
                        wg.BoundedFloatText(
                            value=0.4,
                            layout=wg.Layout(width='5%'),
                            max=1.0,
                            min=-1.0,
                            step=0.01,
                        ),
                        wg.BoundedFloatText(
                            value=0.6,
                            layout=wg.Layout(width='5%'),
                            max=1.0,
                            min=-1.0,
                            step=0.01,
                        ),
                        wg.BoundedFloatText(
                            value=0.45,
                            layout=wg.Layout(width='5%'),
                            max=1.0,
                            min=-1.0,
                            step=0.01,
                        ),
                        wg.BoundedFloatText(
                            value=0.45,
                            layout=wg.Layout(width='5%'),
                            max=1.0,
                            min=-1.0,
                            step=0.01,
                        ),
                        wg.BoundedFloatText(
                            value=-0.05,
                            layout=wg.Layout(width='5%'),
                            max=1.0,
                            min=-1.0,
                            step=0.01,
                        ),
                        wg.BoundedFloatText(
                            value=1.0,
                            layout=wg.Layout(width='5%'),
                            max=1.0,
                            min=-1.0,
                            step=0.01,
                        ),
                        wg.BoundedFloatText(
                            value=0.0,
                            layout=wg.Layout(width='5%'),
                            max=1.0,
                            min=-1.0,
                            step=0.01,
                        ),
                    )
                ),
                wg.HBox(
                    children=(
                        wg.Button(
                            description='Cash',
                            disabled=True,
                            button_style='',
                            tooltip='',
                            icon='',
                            layout=wg.Layout(width='5%'),
                        ),
                        wg.BoundedFloatText(
                            value=0.0,
                            layout=wg.Layout(width='5%'),
                            max=1.0,
                            min=-1.0,
                            step=0.01,
                        ),
                        wg.BoundedFloatText(
                            value=0.0,
                            layout=wg.Layout(width='5%'),
                            max=1.0,
                            min=-1.0,
                            step=0.01,
                        ),
                        wg.BoundedFloatText(
                            value=0.0,
                            layout=wg.Layout(width='5%'),
                            max=1.0,
                            min=-1.0,
                            step=0.01,
                        ),
                        wg.BoundedFloatText(
                            value=0.0,
                            layout=wg.Layout(width='5%'),
                            max=1.0,
                            min=-1.0,
                            step=0.01,
                        ),
                        wg.BoundedFloatText(
                            value=0.0,
                            layout=wg.Layout(width='5%'),
                            max=1.0,
                            min=-1.0,
                            step=0.01,
                        ),
                        wg.BoundedFloatText(
                            value=0.0,
                            layout=wg.Layout(width='5%'),
                            max=1.0,
                            min=-1.0,
                            step=0.01,
                        ),
                        wg.BoundedFloatText(
                            value=0.0,
                            layout=wg.Layout(width='5%'),
                            max=1.0,
                            min=-1.0,
                            step=0.01,
                        ),
                        wg.BoundedFloatText(
                            value=0.0,
                            layout=wg.Layout(width='5%'),
                            max=1.0,
                            min=-1.0,
                            step=0.01,
                        ),
                        wg.BoundedFloatText(
                            value=0.0,
                            layout=wg.Layout(width='5%'),
                            max=1.0,
                            min=-1.0,
                            step=0.01,
                        ),
                        wg.BoundedFloatText(
                            value=1.0,
                            layout=wg.Layout(width='5%'),
                            max=1.0,
                            min=-1.0,
                            step=0.01,
                        ),
                    )
                ),
            )
        )
    )

    corr_links = []
    for i in range(asset_count):
        for j in range(asset_count):
            if i != j:
                corr_links.append(
                    wg.link(
                        (int_corr_matrix.children[i + 1].children[j + 1], 'value'),
                        (int_corr_matrix.children[j + 1].children[i + 1], 'value'),
                    )
                )
    name_links = []
    for i in range(asset_count):
        name_links.append(
            wg.link(
                (assets.children[i + 1].children[0], 'value'),
                (int_corr_matrix.children[0].children[i + 1], 'description'),
            )
        )
        name_links.append(
            wg.link(
                (assets.children[i + 1].children[0], 'value'),
                (int_corr_matrix.children[i + 1].children[0], 'description'),
            )
        )
    for i in range(asset_count):
        for j in range(asset_count):
            if j >= i:
                int_corr_matrix.children[i + 1].children[j + 1].disabled = True

    val_observe = []
    for i in range(asset_count):
      for j in range(asset_count):
        val_observe.append(int_corr_matrix.children[i+1].children[j+1].observe(on_val_change))

    for i in range(asset_count):
      for j in range(4):
        val_observe.append(assets.children[i+1].children[j+1].observe(on_val_change))

    if asset_count > 2:
        rem_btn.disabled = False
    else:
        rem_btn.disabled = True
    
    btn_weights.disabled=True


add_btn.on_click(on_add_click)
rem_btn.on_click(on_rem_click)
default_btn.on_click(on_default_click)

btn = wg.HBox([add_btn, rem_btn, default_btn])
display(assets, new_asset, btn)


## Correlation Matrix
Adjust values of the correlation matrix' lower triangular to update correlations between asset classes.  By default the diagonal is locked at 1 and the upper triangular will take the value of the lower triangular.

In [3]:
display(int_corr_matrix)


## Solving for Optimal Portfolio
Click on 'Fit Portfolio' to compute the efficient frontier implied the return assumptions you input above. Use 'Optimize Weights' to compute the portfolio your target portfolio return or risk, set using the slider.  Press the button `Return` (or `Risk`) to switch between return and risk targets.  When the button says `Return`, the weights will be optimized for the target return value and vice versa.

In [4]:
out_port_panel = wg.Output()
btn_weights.disabled=True

ef_min_ret = 0.0
ef_max_ret = 100.0
ef_val_ret = (ef_min_ret + ef_max_ret) / 2

ef_min_vol = 0.0
ef_max_vol = 100.0
ef_val_vol = (ef_min_vol + ef_max_vol) / 2

ret_slider = wg.FloatSlider(
    value=ef_val_ret,
    min=ef_min_ret,
    max=ef_max_ret,
    step=0.001,
    description='Target:',
    style={'description_width': 'initial'},
)

opt_type_toggle = wg.Button(
    description='Return',
    button_style="",
)
opt_type_toggle.style.button_color = "Crimson"

def on_switch(change):
    if opt_type_toggle.description == "Risk":
      ret_slider.min = 0.0
      ret_slider.max = 100.0
      ret_slider.value = 50.0
      ret_slider.min = ef_min_ret * 100
      ret_slider.max = ef_max_ret * 100
      ret_slider.value = ef_val_ret * 100
      opt_type_toggle.description = "Return"
    elif opt_type_toggle.description == "Return":
      ret_slider.min = 0.0
      ret_slider.max = 100.0
      ret_slider.value = 50.0
      ret_slider.min = ef_min_vol * 100
      ret_slider.max = ef_max_vol * 100
      ret_slider.value = ef_val_vol * 100
      opt_type_toggle.description = "Risk"

opt_type_toggle.on_click(on_switch)

corr_matrix = 0
cov_matrix = 0
exp_returns = 0
stdevs = 0
bounds = 0
asset_names = 0

def on_click_port_plot(x):
    global ef_min_ret, ef_max_ret, ef_val_ret, ef_min_vol, ef_max_vol, ef_val_vol, corr_matrix, cov_matrix, exp_returns, stdevs, bounds, asset_names
    with out_port_panel:
        clear_output(True)

        corr_matrix = np.zeros((asset_count, asset_count))
        for i in range(asset_count):
            for j in range(asset_count):
                corr_matrix[i, j] = (
                    int_corr_matrix.children[i + 1].children[j + 1].value
                )
        cov_matrix = np.zeros((asset_count, asset_count))
        for row in range(asset_count):
            cov_matrix[row, row] = assets.children[row + 1].children[2].value / 100
        cov_matrix = cov_matrix @ corr_matrix @ cov_matrix
        cov_matrix[cov_matrix == 0] = np.finfo(np.float32).tiny
        exp_returns = np.zeros((asset_count, 1))
        for row in range(asset_count):
            exp_returns[row, 0] = (
                assets.children[row + 1].children[1].value / 100
                + (1 / 2) * (assets.children[row + 1].children[2].value / 100) ** 2
            )
        stdevs = np.zeros((asset_count, 1))
        for row in range(asset_count):
            stdevs[row, 0] = (
                assets.children[row + 1].children[2].value / 100
            )
        asset_names = [
            assets.children[row + 1].children[0].value for row in range(asset_count)
        ]

        bounds = [
            (
                assets.children[row + 1].children[3].value / 100,
                assets.children[row + 1].children[4].value / 100,
            )
            for row in range(asset_count)
        ]

        ef = efficient_frontier.EfficientFrontier(
            exp_returns, cov_matrix, weight_bounds=bounds
        )
        ef.tickers = asset_names

        btn_weights.disabled=False

        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 10))
        plotting.plot_efficient_frontier(copy.deepcopy(ef), ax=ax1, show_assets=False)
        ax1.scatter(
            np.sqrt(np.diag(ef.cov_matrix)),
            ef.expected_returns,
            s=30,
            c="white",
            label="Assets",
        )
        ax1.legend()

        labels = [name for name in asset_names]
        vals = [0.0 for i in range(len(asset_names))]
        y_pos = np.arange(len(labels))
        ax2.barh(y_pos, vals)
        xmin, xmax = ax2.get_xlim()
        ax2.set_xlim(xmin * 2.0, xmax * 2.0)
        ax2.set_xlabel("Weight")
        ax2.set_yticks(y_pos)
        ax2.set_yticklabels(labels)
        ax2.invert_yaxis()
        # plotting.plot_weights(ef_w.efficient_return(ret_slider.value / 100), ax=ax2)
        plt.tight_layout()
        plt.show()

        #try:
        ef_min_ret = ax1.lines[0].get_ydata()[0]
        ef_max_ret = ax1.lines[0].get_ydata()[-1]
        ef_val_ret = (ef_min_ret + ef_max_ret) / 2.0
        ef_min_vol = np.sqrt(1 / np.sum(np.linalg.inv(ef.cov_matrix)))
        ef_max_vol = ax1.lines[0].get_xdata()[-1]
        ef_val_vol = (ef_min_vol + ef_max_vol) / 2.0
        ret_slider.min = 0.0
        ret_slider.max = 100.0
        ret_slider.value = 50.0
        if opt_type_toggle.description == "Return":
          ret_slider.min = ef_min_ret * 100
          ret_slider.max = ef_max_ret * 100
          ret_slider.value = ef_val_ret * 100
        elif opt_type_toggle.description == "Risk":
          ret_slider.min = ef_min_vol * 100
          ret_slider.max = ef_max_vol * 100
          ret_slider.value = ef_val_vol * 100
        #except:
        #  print("Solver failed to converge.  Please try adjusting portfolio components.")
        #  btn_weights.disabled=True

def on_click_weights(x):
    global ef_min_ret, ef_max_ret, ef_val_ret, ef_min_vol, ef_max_vol, ef_val_vol
    with out_port_panel:
        clear_output(True)

        ef = efficient_frontier.EfficientFrontier(
            exp_returns, cov_matrix, weight_bounds=bounds
        )
        ef.tickers = asset_names
        ef_w = copy.deepcopy(ef)

        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 10))
        plotting.plot_efficient_frontier(copy.deepcopy(ef), ax=ax1, show_assets=False)
        ax1.scatter(
            np.sqrt(np.diag(ef.cov_matrix)),
            ef.expected_returns,
            s=30,
            c="white",
            label="Assets",
        )
        if opt_type_toggle.description == "Return":
            ef.efficient_return(ret_slider.value / 100)
        elif opt_type_toggle.description == "Risk":
            ef.efficient_risk(ret_slider.value / 100)
        ax1.scatter(
            ef.portfolio_performance()[1],
            ef.portfolio_performance()[0],
            s=30,
            c="red",
            label="Optimal Portfolio",
        )

        points = ax1.collections[1]
        points = points.get_offsets().data[0]
        ax1.annotate(
            f"{'Return: '} {round(points[1]*100, 1)}\n"
            f"{'Volatility: '} {round(points[0]*100, 1)}",
            xy=(1, 0),
            xycoords="axes fraction",
            horizontalalignment="right",
            verticalalignment="bottom",
            fontsize=24,
        )
        ax1.legend()

        ax1.legend()

        if opt_type_toggle.description == "Return":
          desc = ef_w.efficient_return(ret_slider.value / 100).items()
        elif opt_type_toggle.description == "Risk":
          desc = ef_w.efficient_risk(ret_slider.value / 100).items()
        labels = [i[0] for i in desc]
        vals = [i[1] * 100 for i in desc]
        y_pos = np.arange(len(labels))
        ax2.barh(y_pos, vals)
        ax2.bar_label(ax2.containers[0], label_type='edge', fmt="%10.1f%%")
        xmin, xmax = ax2.get_xlim()
        ax2.set_xlim(xmin * 2.0, xmax * 2.0)
        ax2.set_xlabel("Weight")
        ax2.set_yticks(y_pos)
        ax2.set_yticklabels(labels)
        ax2.invert_yaxis()
        plt.tight_layout()

        plt.show()

btn_port_plot.on_click(on_click_port_plot)
btn_weights.on_click(on_click_weights)

opt_term = wg.HBox([opt_type_toggle, ret_slider])

port_inputs = wg.HBox([btn_port_plot, btn_weights, opt_term])
port_display = wg.VBox([port_inputs, out_port_panel])
display(port_display)
